In [1]:
include("../src/viral_load_infectivity_testpos.jl")
using StatsPlots

In [2]:
function calc_infectious_potential_reduction(test_days::Array{Int64,1}, day_start::Int64,
            VL_profile::Array{Float64,1}, VL_mag::Float64, test_pos_profile::Array{Float64,1},
            will_isolate::Bool, symp_day::Int64, NCP::Float64, delay::Int64 = 0)
    
    test_daysh = test_days .+ day_start
    #trim to test days within VL trajectory
    valid_days = test_daysh[test_daysh .< length(VL_profile)]
    #baseline infection profile
    inf_profile0 = 2 * VL_profile / VL_mag
    inf_profile0[inf_profile0 .< 0] .= 0
    inf_profile = copy(inf_profile0)
    testneg = true  #repeat tests until we get a positive (then assumed quarantined)
    itest = 1
    while testneg == true && itest <= length(valid_days)
        d = valid_days[itest]
        tp = (rand() < NCP*test_pos_profile[d])  #assume non compliance is independent each time
        if tp && (d+delay <= length(inf_profile))
            inf_profile[(d+delay):end] .= 0
            testneg = false
        end
        itest += 1
    end
    if will_isolate   #symptomatic isolation alters both tested and original profile
        inf_profile[(symp_day+1):end] .= 0
        inf_profile0[(symp_day+1):end] .= 0
    end
    if sum(inf_profile0) > 0
        p_infected = 1 - (sum(inf_profile) / sum(inf_profile0))   #reduction in infectious potential
    else
        p_infected = 0  #should be rare
    end
    
    return p_infected
end

calc_infectious_potential_reduction (generic function with 2 methods)

In [3]:
Ntrials = 50000

50000

Testing with no symptomatic self isolation -- LFDs


In [4]:
Pisol = 0.0
NewPisol = [0.25, 0.5,0.75,1.0]  #adherence to test result
TestFreq = 2:2:14
p_reduction = zeros(length(NewPisol),length(TestFreq),Ntrials)
mean_p_reduction = zeros(length(NewPisol),length(TestFreq))
std_p_reduction = zeros(length(NewPisol),length(TestFreq))
for (j,p) in enumerate(NewPisol)
    for (n,f) in enumerate(TestFreq)
        sim = init_VL_and_infectiousness(Ntrials, Pisol)
        testing_params = Dict("protocol"=>LFD_mass_protocol,"tperiod"=>f,
                              "new_comply_prob"=>p)
        init_testing!(sim, testing_params, 1, 50)
        test_days = collect(1:f:50)
        #work out probability of avoiding infection for each case
        for m in 1:Ntrials
            day_start = rand(0:(f-1))   
            p_reduction[j,n,m] = calc_infectious_potential_reduction(test_days, day_start,
                       sim["VL_profiles"][m], sim["VL_mag"][m], 
                       sim["test_pos_profiles"][m], sim["will_isolate"][m], 
                       sim["symp_day"][m], p)
        end
        mean_p_reduction[j,n] = mean(p_reduction[j,n,:])
        std_p_reduction[j,n] = std(p_reduction[j,n,:])
    end
end
dftest_noisol = DataFrame([Float64,Float64,Int64,Float64],[:TestFreq,:NCP,:Iter,:p_inf],length(p_reduction))
n = 1
for i in 1:length(TestFreq)
    for j in 1:length(NewPisol)
        dftest_noisol["TestFreq"][n:(n+Ntrials-1)] .= TestFreq[i]
        dftest_noisol["NCP"][n:(n+Ntrials-1)] .= NewPisol[j]
        dftest_noisol["Iter"][n:(n+Ntrials-1)] = collect(1:Ntrials)
        dftest_noisol["p_inf"][n:(n+Ntrials-1)] .= p_reduction[j,i,:]
        n += Ntrials
    end
end

LoadError: [91mMethodError: no method matching ^(::Float64, ::Array{Float64,1})[39m
[91m[0mClosest candidates are:[39m
[91m[0m  ^(::Float64, [91m::Float64[39m) at math.jl:885[39m
[91m[0m  ^(::Float64, [91m::Integer[39m) at math.jl:899[39m
[91m[0m  ^(::T, [91m::Complex{T}[39m) where T<:Real at complex.jl:783[39m
[91m[0m  ...[39m

In [5]:
colors = Array{RGBA{Float64}}(undef,length(p_reduction))
barcolor=[RGBA(0.1, 0.4, 1.0, 0.25), RGBA(0.1, 0.4, 1.0, 0.5),
          RGBA(0.1, 0.4, 1.0, 0.75), RGBA(0.1, 0.4, 1.0, 1.0)]
i = 1
for ncp in NewPisol
    colors[dftest_noisol["NCP"] .== ncp] .= barcolor[i]
    i += 1
end

gr(size=(800,300))
p = @df dftest_noisol StatsPlots.groupedboxplot(:TestFreq,:p_inf,group=:NCP,ylim=(-0.01,1.01),
         outliers=false, xlabel="Days between tests",
         ylabel="Reduction in infection potential (p)", 
         color = colors,
         xticks=2:2:14,yticks=0:0.1:1.0,margin=5Plots.mm)
savefig(p,"compliance_effect_Pisol0_domcare.png")

mstyles = [:cross, :xcross, :rect, :circle]
for j in 1:length(NewPisol)
    labelh = @sprintf "%.2f" NewPisol[j]
    if j == 1
        Plots.plot(TestFreq .- (0.25 - 0.1*j), mean_p_reduction[j,:], yerror=std_p_reduction[j,:], 
                   color=barcolor[j], markershape=mstyles[j], markerfacecolor=barcolor[j], 
                   markerstrokecolor=barcolor[j], linewidth=2, markerstrokewidth=2, label = labelh)
    else
        Plots.plot!(TestFreq .- (0.25 - 0.1*j), mean_p_reduction[j,:], yerror=std_p_reduction[j,:], 
                    color=barcolor[j], markerstrokecolor=barcolor[j],
                    linewidth=2, markerstrokewidth=2, label = labelh, markershape=mstyles[j], 
                    markerfacecolor=barcolor[j], xticks=2:2:14, ylim=(0,1))
    end
end

Plots.savefig("compliance_effect_Pisol0_domcare_mean_std.png")

LoadError: [91mUndefVarError: RGBA not defined[39m

In [6]:
#lfd_testing -- with isolation
#pcr_testing -- with delay
Pisol = 0.0
TestFreq = 2:2:14
TestTypes = ["LFD","PCR - 0 day delay","PCR - 1 day delay","PCR - 2 day delay"]
p_reduction = zeros(length(TestTypes),length(TestFreq),Ntrials)
mean_p_reduction = zeros(length(TestTypes),length(TestFreq))
std_p_reduction = zeros(length(TestTypes),length(TestFreq))
VLs = zeros(length(TestFreq),Ntrials)
for (n,f) in enumerate(TestFreq)
    simLFD = sim = init_VL_and_infectiousness(Ntrials, Pisol)
    testing_paramsLFD = Dict("protocol"=>LFD_mass_protocol,"tperiod"=>f,
                              "new_comply_prob"=>1.0)
    simPCR = copy(simLFD)  #do for same individuals
    
    init_testing!(simLFD, testing_paramsLFD, 1, 50)
    testing_paramsPCR = Dict("protocol"=>PCR_mass_protocol,"tperiod"=>f,
                          "new_comply_prob"=>1.0)
    init_testing!(simPCR, testing_paramsPCR, 1, 50)
    test_days = collect(1:f:50)
    
    #work out probability of avoiding infection for each case
    for m in 1:Ntrials
        day_start = rand(0:(f-1))
        p_reduction[1,n,m] = calc_infectious_potential_reduction(test_days, day_start,
                       simLFD["VL_profiles"][m], simLFD["VL_mag"][m], 
                       simLFD["test_pos_profiles"][m], simLFD["will_isolate"][m], 
                       simLFD["symp_day"][m], 1.0)
        p_reduction[2,n,m] = calc_infectious_potential_reduction(test_days, day_start,
                       simPCR["VL_profiles"][m], simPCR["VL_mag"][m], 
                       simPCR["test_pos_profiles"][m], simPCR["will_isolate"][m], 
                       simPCR["symp_day"][m], 1.0, 0)
        p_reduction[3,n,m] = calc_infectious_potential_reduction(test_days, day_start,
                       simPCR["VL_profiles"][m], simPCR["VL_mag"][m], 
                       simPCR["test_pos_profiles"][m], simPCR["will_isolate"][m], 
                       simPCR["symp_day"][m], 1.0, 1)
        p_reduction[4,n,m] = calc_infectious_potential_reduction(test_days, day_start,
                       simPCR["VL_profiles"][m], simPCR["VL_mag"][m], 
                       simPCR["test_pos_profiles"][m], simPCR["will_isolate"][m], 
                       simPCR["symp_day"][m], 1.0, 2)
        VLs[n,m] = simLFD["VL_mag"][m]
    end
    for k in 1:length(TestTypes)
        mean_p_reduction[k,n] = mean(p_reduction[k,n,:])
        std_p_reduction[k,n] = std(p_reduction[k,n,:]) 
    end
end
dftest_noisol_LP = DataFrame([Float64,String,Int64,Float64,Float64],[:TestFreq,:TestType,:Iter,:p_inf,:peakVL],
              length(p_reduction))

n = 1
for j in 1:length(TestTypes)
    for i in 1:length(TestFreq)
        dftest_noisol_LP["TestFreq"][n:(n+Ntrials-1)] .= TestFreq[i]
        dftest_noisol_LP["TestType"][n:(n+Ntrials-1)] .= TestTypes[j]
        dftest_noisol_LP["Iter"][n:(n+Ntrials-1)] .= collect(1:Ntrials)
        dftest_noisol_LP["p_inf"][n:(n+Ntrials-1)] .= p_reduction[j,i,:]
        dftest_noisol_LP["peakVL"][n:(n+Ntrials-1)] .= VLs[i,:]
        n += Ntrials
    end
end

LoadError: [91mUndefVarError: init_VL_and_infectiousness not defined[39m

In [7]:
colors = Array{RGBA{Float64}}(undef,length(p_reduction))
barcolor=[RGBA(0.1, 0.4, 1.0, 1.0), RGBA(1.0, 0.55, 0.0, 1.0), 
         RGBA(1.0, 0.55, 0.0, 0.6), RGBA(1.0, 0.55, 0.0, 0.2)]
i = 1
for tt in TestTypes
    colors[dftest_noisol_LP["TestType"] .== tt] .= barcolor[i]
    i += 1
end

gr(size=(800,300))
plot = @df dftest_noisol_LP StatsPlots.groupedboxplot(:TestFreq,:p_inf,group=:TestType,ylim=(-0.01,1.01),
         outliers=false, xlabel="Days between tests",
         ylabel="Reduction in infection potential (p)",
         color = colors,
         xticks=2:2:14,yticks=0:0.1:1.0, margin=5Plots.mm)
savefig(plot,"test_effect_Pisol0_domcare.png")

mstyles = [:circle, :xcross, :rect, :cross]
for j in 1:length(TestTypes)
    if j == 1
        Plots.plot(TestFreq .- (0.25 - 0.1*j), mean_p_reduction[j,:], yerror=std_p_reduction[j,:], 
                   color=barcolor[j], markershape=mstyles[j], markerfacecolor=barcolor[j], 
                   markerstrokecolor=barcolor[j], linewidth=2, markerstrokewidth=2, label = TestTypes[j])
    else
        Plots.plot!(TestFreq .- (0.25 - 0.1*j), mean_p_reduction[j,:], yerror=std_p_reduction[j,:], 
                    color=barcolor[j], markerstrokecolor=barcolor[j],
                    linewidth=2, markerstrokewidth=2, label = TestTypes[j], markershape=mstyles[j], 
                    markerfacecolor=barcolor[j], xticks=2:2:14, ylim=(0,1))
    end
end

Plots.savefig("test_effect_Pisol0_domcare_mean_std.png")

LoadError: [91mUndefVarError: RGBA not defined[39m

In [8]:
#lfd_testing -- with isolation
#pcr_testing -- with delay
Pisol = 1.0
TestFreq = 2:2:14
TestTypes = ["LFD","PCR - 0 day delay","PCR - 1 day delay","PCR - 2 day delay"]
p_reduction = zeros(length(TestTypes),length(TestFreq),Ntrials)
mean_p_reduction = zeros(length(TestTypes),length(TestFreq))
std_p_reduction = zeros(length(TestTypes),length(TestFreq))
VLs = zeros(length(TestFreq),Ntrials)
for (n,f) in enumerate(TestFreq)
    simLFD = sim = init_VL_and_infectiousness(Ntrials, Pisol)
    testing_paramsLFD = Dict("protocol"=>LFD_mass_protocol,"tperiod"=>f,
                              "new_comply_prob"=>1.0)
    simPCR = copy(simLFD)  #do for same individuals
    
    init_testing!(simLFD, testing_paramsLFD, 1, 50)
    testing_paramsPCR = Dict("protocol"=>PCR_mass_protocol,"tperiod"=>f,
                          "new_comply_prob"=>1.0)
    init_testing!(simPCR, testing_paramsPCR, 1, 50)
    test_days = collect(1:f:50)
    
    #work out probability of avoiding infection for each case
    for m in 1:Ntrials
        day_start = rand(0:(f-1))
        p_reduction[1,n,m] = calc_infectious_potential_reduction(test_days, day_start,
                       simLFD["VL_profiles"][m], simLFD["VL_mag"][m], 
                       simLFD["test_pos_profiles"][m], simLFD["will_isolate"][m], 
                       simLFD["symp_day"][m], 1.0)
        p_reduction[2,n,m] = calc_infectious_potential_reduction(test_days, day_start,
                       simPCR["VL_profiles"][m], simPCR["VL_mag"][m], 
                       simPCR["test_pos_profiles"][m], simPCR["will_isolate"][m], 
                       simPCR["symp_day"][m], 1.0, 0)
        p_reduction[3,n,m] = calc_infectious_potential_reduction(test_days, day_start,
                       simPCR["VL_profiles"][m], simPCR["VL_mag"][m], 
                       simPCR["test_pos_profiles"][m], simPCR["will_isolate"][m], 
                       simPCR["symp_day"][m], 1.0, 1)
        p_reduction[4,n,m] = calc_infectious_potential_reduction(test_days, day_start,
                       simPCR["VL_profiles"][m], simPCR["VL_mag"][m], 
                       simPCR["test_pos_profiles"][m], simPCR["will_isolate"][m], 
                       simPCR["symp_day"][m], 1.0, 2)
        VLs[n,m] = simLFD["VL_mag"][m]
    end
    for k in 1:length(TestTypes)
        mean_p_reduction[k,n] = mean(p_reduction[k,n,:])
        std_p_reduction[k,n] = std(p_reduction[k,n,:]) 
    end
end
dftest_isol = DataFrame([Float64,String,Int64,Float64,Float64],[:TestFreq,:TestType,:Iter,:p_inf,:peakVL],
              length(p_reduction))

n = 1
for j in 1:length(TestTypes)
    for i in 1:length(TestFreq)
        dftest_isol["TestFreq"][n:(n+Ntrials-1)] .= TestFreq[i]
        dftest_isol["TestType"][n:(n+Ntrials-1)] .= TestTypes[j]
        dftest_isol["Iter"][n:(n+Ntrials-1)] .= collect(1:Ntrials)
        dftest_isol["p_inf"][n:(n+Ntrials-1)] .= p_reduction[j,i,:]
        dftest_isol["peakVL"][n:(n+Ntrials-1)] .= VLs[i,:]
        n += Ntrials
    end
end

LoadError: [91mUndefVarError: init_VL_and_infectiousness not defined[39m

In [9]:
colors = Array{RGBA{Float64}}(undef,length(p_reduction))
barcolor=[RGBA(0.1, 0.4, 1.0, 1.0), RGBA(1.0, 0.55, 0.0, 1.0), 
         RGBA(1.0, 0.55, 0.0, 0.6), RGBA(1.0, 0.55, 0.0, 0.2)]
i = 1
for tt in TestTypes
    colors[dftest_isol["TestType"] .== tt] .= barcolor[i]
    i += 1
end

gr(size=(800,300))
plot = @df dftest_isol StatsPlots.groupedboxplot(:TestFreq,:p_inf,group=:TestType,ylim=(-0.01,1.01),
         outliers=false, xlabel="Days between tests",
         ylabel="Reduction in infection potential (p)",
         color = colors,
         xticks=2:2:14,yticks=0:0.1:1.0, margin=5Plots.mm)
savefig(plot,"test_effect_Pisol1_domcare.png")

mstyles = [:circle, :xcross, :rect, :cross]
for j in 1:length(TestTypes)
    if j == 1
        Plots.plot(TestFreq .- (0.25 - 0.1*j), mean_p_reduction[j,:], yerror=std_p_reduction[j,:], 
                   color=barcolor[j], markershape=mstyles[j], markerfacecolor=barcolor[j], 
                   markerstrokecolor=barcolor[j], linewidth=2, markerstrokewidth=2, label = TestTypes[j])
    else
        Plots.plot!(TestFreq .- (0.25 - 0.1*j), mean_p_reduction[j,:], yerror=std_p_reduction[j,:], 
                    color=barcolor[j], markerstrokecolor=barcolor[j],
                    linewidth=2, markerstrokewidth=2, label = TestTypes[j], markershape=mstyles[j], 
                    markerfacecolor=barcolor[j], xticks=2:2:14, ylim=(0,1))
    end
end

Plots.savefig("test_effect_Pisol1_domcare_mean_std.png")

LoadError: [91mUndefVarError: RGBA not defined[39m

In [10]:
gr(size=(1200,400))
dftest_isol_LFD = dftest_isol[dftest_isol["TestType"].=="LFD",:]
dftest_isol_LFD["ViralLoad"] = Array{String}(undef,nrow(dftest_isol_LFD))

dftest_isol_LFD[dftest_isol_LFD["peakVL"].<6,"ViralLoad"] .= "VL low (<6)"
dftest_isol_LFD[(dftest_isol_LFD["peakVL"].>6) .* (dftest_isol_LFD["peakVL"].<9) ,"ViralLoad"] .= "VL med (6-9)"
dftest_isol_LFD[dftest_isol_LFD["peakVL"].>9,"ViralLoad"] .= "VL upp (>9)"
plot = @df dftest_isol_LFD StatsPlots.groupedboxplot(:TestFreq,:p_inf,group=:ViralLoad,ylim=(-0.01,1.01),
         outliers=false, xlabel="Days between tests",
         ylabel="Reduction in infection potential (p)",
         xticks=1:14,yticks=0:0.1:1.0, margin=5Plots.mm)
         savefig(plot,"test_effect_Pisol1_domcare_VLdep.png")

LoadError: [91mUndefVarError: gr not defined[39m